In [36]:
!pip install streamlit --quiet
!pip install schedule


In [37]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import streamlit as st
import joblib

In [ ]:
import requests
import pandas as pd
import schedule
import time

api_endpoint = 'https://scm-backend-test.ienetworks.co/api/scm/stock/inventory/index'

headers = {
    'Content-Type': 'application/json',
}

def fetch_data():
    try:
        response = requests.get(api_endpoint, headers=headers, timeout=10)
        response.raise_for_status()  # Raises HTTPError if the response code is 4xx/5xx
        
        data2 = response.json()
        
        # Check if 'data' exists in the response and handle accordingly
        if 'data' in data2:
            # Create DataFrame
            df = pd.DataFrame(data2['data'])
            
            if not df.empty:
                print("Data fetched successfully!")
                print(df)  # Display or do something with the DataFrame
            else:
                print("No data found in the response.")
        else:
            print("'data' key not found in response.")
    
    except requests.exceptions.RequestException as e:
        print(f'Failed to retrieve data: {e}')

# Schedule the fetch_data function to run every 10 seconds
# You can adjust the frequency as per your requirements
schedule.every(10).seconds.do(fetch_data)

print("Scheduler started. Press Ctrl+C to stop.")

# Keep the script running
try:
    while True:
        schedule.run_pending()
        time.sleep(1)  # Sleep to avoid busy waiting
except KeyboardInterrupt:
    print("Scheduler stopped.")

Scheduler started. Press Ctrl+C to stop.
Data fetched successfully!
                                        id  \
0     0003e313-bc27-4c94-ac3b-21b08383c3fa   
1     00127af5-67f8-4819-a7e2-6fa0a225611f   
2     00247031-422e-4a20-b21a-c1dd044ff11d   
3     0038b2e0-5fd7-46b7-84e6-7a102a852975   
4     003bb89a-bb28-4220-b223-eec0bda808e6   
...                                    ...   
6133  ffdf2f15-cb88-4254-b565-4115ee31c855   
6134  ffe4dd8e-8757-4b92-803c-36633cf396fb   
6135  fff18592-40f8-41bd-8f17-2f6515ffe601   
6136  fffaab7f-b1b1-4404-abf6-c1fd923231c2   
6137  fffe79a6-4dd7-4a64-aa69-8ecad9d21a45   

                                              item_name  \
0                          DELL EMC VMAX STORAGE SERVER   
1     Edge L angle frame plastic L angle for edge ch...   
2     Edge L angle frame plastic L angle for edge ch...   
3                                      Labeller casette   
4     L angle white Aluminum  L angle for LED light ...   
...                      

In [ ]:
print(data2.columns)

In [ ]:
data2.shape

In [ ]:
print(data2.isnull().sum())

In [ ]:
threshold = 5000

# Drop columns where the number of null values exceeds the threshold
data2= data2.dropna(axis=1, thresh=len(data2) - threshold)

# Display the DataFrame after dropping columns
print("\nDataFrame after dropping columns with more than 5000 null values:")
display(data2)

In [ ]:
print(data2.isnull().sum())

In [ ]:
data2.shape

In [ ]:
print(data2.dtypes)

In [ ]:
columns_to_drop = ['store', 'uom']
data2 = data2.drop(columns=columns_to_drop)

In [ ]:
data2.shape

In [ ]:
print(data2.isnull().sum())

In [ ]:
data2['quantity'] = data2['quantity'].fillna(data2['quantity'].mean())
data2['price'] = data2['price'].fillna(data2['price'].median())
categorical_features = ['uom_id','department_id','confirmed_by','created_by','description','po_id', 'project_id']
for feature in categorical_features:
    data2[feature] = data2[feature].fillna('Missing')
data2 = data2.dropna(subset=['date_of_purchased'])

In [ ]:
print(data2.isnull().sum())

In [ ]:
data2['is_returned'].value_counts()

In [ ]:
display(data2.head())

In [ ]:
data2['is_consumable'].value_counts()

In [ ]:
data2['project_id'].value_counts()

In [ ]:
data2['item_name'].value_counts()

In [ ]:
print(data2.columns)

In [ ]:
# Count items per project
project_item_counts = data2['project_id'].value_counts().reset_index()
project_item_counts.columns = ['project_id', 'item_count']

print(project_item_counts)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Assuming `data2` is your DataFrame

# Visualize Items per Department
plt.figure(figsize=(20, 6))
sns.histplot(data2['department_id'], bins=30)

# Set diagonal labels for x-axis
plt.xticks(rotation=45, ha='right')  # Rotate labels for better readability
plt.title('Items per Department')
plt.xlabel('Department ID')  # Optional: Add an x-axis label
plt.ylabel('Count')  # Optional: Add a y-axis label
plt.show()



In [ ]:
# Visualize Price Distribution
plt.figure(figsize=(10, 6))
sns.histplot(data2['price'], bins=30)

# Set diagonal labels for x-axis
  # Rotate labels for better readability
plt.title('Price Distribution')
plt.xlabel('Price')  # Optional: Add an x-axis label
plt.ylabel('Count')  # Optional: Add a y-axis label
plt.show()

In [ ]:
display(data2[['date_of_purchased','amount','created_at','updated_at', 'amount','quantity']])

In [ ]:
data2.to_csv('processed_data.csv', index=False)

In [ ]:
import streamlit as st
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from datetime import datetime


# Encode categorical data
le_item = LabelEncoder()
data2['item_name_encoded'] = le_item.fit_transform(data2['item_name'])

# Creating features and labels
X = data2[['item_name_encoded', 'quantity']]
y = data2['project_id']

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print(f"Model Accuracy: {accuracy}")


In [ ]:
import joblib
from sklearn.preprocessing import LabelEncoder


le_item = LabelEncoder()

le_item.fit(data2['item_name'])
joblib.dump(le_item, 'label_encoder.pkl')

In [ ]:
import joblib
joblib.dump(model, 'model.pkl')
